In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/stock_prediction

/content/gdrive/MyDrive/stock_prediction


In [ ]:
import os 
import torch 
import torch.nn as nn
import pandas as pd 
import numpy as np 
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(1)
torch.cuda.manual_seed(1)

In [ ]:
train_df = pd.read_csv('./Data/preprocessed_etf/train_with_col_id.csv')
test_df = pd.read_csv('./Data/preprocessed_etf/test_with_col_id.csv')

In [ ]:
train_df

,date,spy_open,spy_high,spy_low,spy_close,ivv_open,ivv_high,ivv_low,ivv_close,vti_open,vti_high,vti_low,vti_close,qqq_open,qqq_high,qqq_low,qqq_close,agg_open,agg_high,agg_low,agg_close
0,2005-02-25,104.77,106.00,104.68,105.79,104.84,105.91,104.78,105.84,51.657,52.187,51.638,52.143,33.755,34.045,33.675,33.972,86.493,86.516,86.295,86.516
1,2005-02-28,105.55,105.68,104.56,105.08,105.56,105.78,104.72,105.15,52.143,52.153,51.629,51.876,33.831,33.959,33.343,33.614,86.516,86.534,86.163,86.287
2,2005-03-01,105.22,105.87,105.22,105.62,105.37,105.91,105.37,105.74,51.966,52.235,51.966,52.180,33.747,34.046,33.711,33.982,85.846,86.050,85.846,85.985
3,2005-03-02,105.21,106.22,105.10,105.57,105.29,106.24,105.20,105.71,51.893,52.431,51.893,52.124,33.792,34.343,33.746,33.972,85.779,86.016,85.779,85.874
4,2005-03-03,105.99,106.20,105.15,105.61,106.02,106.23,105.25,105.73,52.229,52.390,51.902,52.187,34.060,34.078,33.437,33.690,86.060,86.069,85.732,85.943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978,2016-12-23,222.26,222.55,222.04,222.54,223.66,223.92,223.43,223.84,114.690,114.880,114.590,114.880,119.120,119.410,118.970,119.360,105.330,105.330,105.180,105.210
2979,2016-12-27,222.85,223.55,222.83,223.10,224.22,224.92,224.22,224.47,115.030,115.440,114.960,115.170,119.600,120.680,119.570,119.980,104.980,105.130,104.980,105.130
2980,2016-12-28,223.40,223.42,221.13,221.26,224.78,224.78,222.50,222.62,115.340,115.380,114.120,114.200,120.220,120.350,118.960,119.040,105.120,105.390,105.090,105.300
2981,2016-12-29,221.34,221.73,220.70,221.21,222.70,223.10,222.08,222.60,114.200,114.490,113.930,114.210,119.020,119.290,118.500,118.880,105.520,105.670,105.440,105.630


In [ ]:
## DataSet 만들기 
class ETFPriceDataSet(Dataset) :
  def __init__(self, file_path, target, ref_days=20)  :
    import pandas as pd 
    self.data = pd.read_csv(file_path)
    self.feature = torch.Tensor(self.data.iloc[:-ref_days, 1:].values)
    
    self.ref_days = ref_days
    self.target_id = self.data.columns.get_loc(target)
    
  def __len__(self) :
    return len(self.data) - (self.ref_days + 1)

  def __getitem__(self, idx) :
    if type(idx) == slice :
      cur_item = [ self.feature[i:i+(self.ref_days+1)] for i in range(idx.start, idx.stop, idx.step if idx.step != None else 1)]
    elif type(idx) == int:
      cur_item = torch.Tensor(self.feature[i:i+(self.ref_days+1)])
    return cur_item

In [ ]:
## DataSet 제대로 되는지 확인 
train_dataset = ETFPriceDataSet('./Data/preprocessed_etf/train_with_col_id.csv',target='spy_close')

for i in range(20,len(train_dataset)) :
  print(i)
  feat = train_dataset[i]

20


In [ ]:
## Data Loader
etf_price_loader = DataLoader(train_dataset, batch_size=4, num_workers=2)

In [ ]:
## DataLoader 제대로 되는지 확인
for cur_id, cur_batch in enumerate(etf_price_loader) : 
  inp = cur_batch[:,:etf_price_loader.dataset.ref_days,:]
  trg = cur_batch[:,-1,etf_price_loader.dataset.target_id]
  print(inp.size())
  print(trg.size())

torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])
torch.Size([4, 20, 20])
torch.Size([4])


In [ ]:
## Time2Vec 

class Time2Vec(nn.Module) : 
  def __init__(self, time_scalar, feature_size, prd_act) :
    super(Time2Vec,self).__init__()

    self.feature_size = feature_size
    self.embed_size = embed_size 
    
    self.init_time_vec = torch.tensor([[time_scalar]], dtype=torch.float32)
    self.lin_comb = nn.Linear(1, feature_size, bias=True)
    self.prd_act = self.get_prd_activation(prd_act)

  def forward(self, x) :
     # |x| = (batch_size, length, feature_size)

     lin_time_vec = self.lin_comb(self.init_time_vec)
     # |lin_time_vec| = (1, feature_size)
     print('lin_time_vec size', lin_time_vec.size())

     time_vec = self.prd_act(lin_time_vec)
     time_vec[0][0] = lin_time_vec[0][0]
     # |time_vec| = (1, feature_size)

     time_vec = time_vec.unsqueeze(0)
     # |time_vec| = (1,1, feautre_size)

     encoded_vector = x + time_vec
     #|embedding_vector| = (batch_size, length, feature_size)

     return encoded_vector

  def get_prd_activation(self,prd_act) :
    if prd_act == 'sin' : 
      return torch.sin
    elif prd_act == 'cosine' :
      return torch.cos

In [ ]:
## Transformer 모델 만들기 
class ETFTransformer(nn.Module) : 
  def __init__(self, input_size, time_scalar, prd_act, d_model, heads_num) : 
    super(ETFTransformer, self).__init__()
 
    # Model의 input vector에 time2vec으로 생성된 시간에 대한 정보를 추가하기 위한 layer
    self.time_enc_layer = Time2Vec(time_scalar, feature_size, prd_act)


  def forward(self, x)



In [ ]:
a=torch.Tensor([[2,1]])

In [ ]:
b = a.unsqueeze(0)

In [ ]:
a.size()

torch.Size([1, 2])

In [ ]:
b.size()

torch.Size([1, 1, 2])

In [ ]:
torch.conc